# Live Audio Visualizer

-  Group ID: **5**
-  Team members: 
    - **Shashank datta Bezgam**
    - **Dev Patel**
-  Due date: **11-29-2022** 
-  Assignment: **4 (Group Project)** 
-  Course: **CS2304**

# Honor Pledge

In [ ]:
#  On my honor: 
#   
#  - I have not discussed the Python code in my program with 
#    anyone other than my instructor or the teaching assistants  
#    assigned to this course. 
#   
#  - I have not used Python code obtained from another student,  
#    or any other unauthorized source, either modified or  
#    unmodified.   
#   
#  - If any Python code or documentation used in my program  
#    was obtained from another source, such as a text book or 
#    course notes, that has been clearly noted with a proper 
#    citation in the comments of my program. 
#   
#   Shashank datta Bezgam
#   Dev Patel

### Necessary Package Imports And Defining Constants

In [ ]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.fftpack import fft
from tkinter import TclError
import pandas as pd
from datetime import datetime

# to display in separate Tk window
%matplotlib tk

# constants
THRESHOLD_X = 2000
THRESHOLD_Y = 0.6
RATE = 44100                 # samples per second
FORMAT = pyaudio.paInt16     
CHANNELS = 1                 # channel for microphone
FILENAME = "output.csv"
CHUNK = 1024 * 2             

### Main Program

In [ ]:
# Creating dataframe for frequency
threshold_df = pd.DataFrame(columns=['Frequency', 'Loudness', 'TimeStamp'])

# create matplotlib figure and axes
fig, (graph_1, graph_2) = plt.subplots(2, figsize=(16, 9))

stream = pyaudio.PyAudio().open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

wave_data = np.arange(0, 2 * CHUNK, 2)       # waveform
freq_data = np.linspace(0, RATE, CHUNK)     # frequencies

# create a wave_graph object with random data
wave_graph, = graph_1.plot(wave_data, np.random.rand(CHUNK), '-', color="blue", lw=2)

# create semilogx wave_graph for spectrum
freq_graph, = graph_2.semilogx(freq_data, np.random.rand(CHUNK), '-', color="blue", lw=2)

# format waveform axes
graph_1.set_title('Audio Visualizer')
graph_1.set_xlabel('Chunks')
graph_1.set_ylabel('Loudness')
graph_1.set_ylim(0, 255)
graph_1.set_xlim(0, 2 * CHUNK)
plt.setp(graph_1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])


graph_2.set_xlabel('Frequency (Log10)')
graph_2.set_ylabel('Loudness')
plt.axhspan(0, THRESHOLD_Y, 0, 0.658, facecolor='green', alpha=0.7)
plt.axhspan(THRESHOLD_Y, 1.0, 0.658, RATE / 2, facecolor='red', alpha=0.7)
plt.axhspan(0, THRESHOLD_Y, 0.658, RATE / 2, facecolor='orange', alpha=0.7)
plt.axhspan(THRESHOLD_Y, 1.0, 0, 0.658, facecolor='yellow', alpha=0.7)

pop_g = mpatches.Patch(color='green', label='NORMAL Noise Levels')
pop_o = mpatches.Patch(color='orange', label='MODERATE Noise Levels')
pop_r = mpatches.Patch(color='red', label='SEVERE Noise Levels')
pop_y = mpatches.Patch(color='yellow', label='SLIGHT Noise Levels')
plt.legend(handles=[pop_g, pop_o, pop_r, pop_y], ncol=4, loc='upper left', fancybox=True, shadow=True)

# format spectrum axes
graph_2.set_xlim(20, RATE / 2)

print('Program is running now')

df_length = 0

while 1:
    
    # binary data
    audio_data = stream.read(CHUNK, exception_on_overflow = False)  
    
    # ofsetting by + B
    audio_data_int = struct.unpack(str(2 * CHUNK) + 'B', audio_data)
    
    # ofsetting back to original
    audio_data_np = np.array(audio_data_int, dtype='b')[::2] + 128

    yf = fft(audio_data_int)
    temp = np.abs(yf[0:CHUNK]) * 2 / (256 * CHUNK)
    
    wave_graph.set_ydata(audio_data_np)
    freq_graph.set_ydata(temp)
    
    (x_freq_graph, y_freq_graph) = freq_graph.get_data()
    x_freq_graph = np.array(x_freq_graph)[10:]
    y_freq_graph = np.array(y_freq_graph)[10:]
    threshold_arr = np.where(np.logical_and(x_freq_graph > THRESHOLD_X, y_freq_graph > THRESHOLD_Y))
    if (len(threshold_arr[0]) > 0 and x_freq_graph[threshold_arr[0][0]] < RATE / 2):
        threshold_df.loc[df_length] = [x_freq_graph[threshold_arr[0][0]], y_freq_graph[threshold_arr[0][0]], datetime.now().strftime("%d/%m/%Y %H:%M:%S")]
        df_length += 1
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    except TclError:
        break

print('Program has stopped')
threshold_df.to_csv(FILENAME, mode='w')